In [ ]:
#Парсинг cian - коммерческие земельные участки

In [2]:
#Импортируем необходиимых библиотек для реализации парсинга
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

In [68]:
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
#https://chromedriver.chromium.org/ - тут можно скачать актуальный chromedriver.exe

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

link_all = []

p = 3 # количество страниц

for page in tqdm(range(1, p, 1), 'Обработанно страниц'):
    
    url = f"https://www.cian.ru/cat.php?cats%5B0%5D=commercialLandSale&deal_type=sale&engine_version=2&offer_type=offices&p={page}&region=4777"
    driver.get(url)
    sleep(2) #спим 2 секунды (прогрузка страницы)
    
    #выгружаем ссылку по всем квартирам
    blocks = driver.find_elements_by_class_name('_32bbee5fda--offer-title-content-container--_kUhi')
    for block in blocks:
        link = block.find_element_by_tag_name('a').get_attribute("href")
        link_all.append(link)

C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2650938130.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
Обработанно страниц:   0%|                                                                       | 0/2 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2650938130.py:28: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  blocks = driver.find_elements_by_class_name('_32bbee5fda--offer-title-content-container--_kUhi')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2650938130.py:30: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  link = block.find_element_by_tag_name('a').get_attribute("href")
Обработанно страниц: 100%|███████████████████████████████████████████████████████████

In [69]:
link_all

['https://kazan.cian.ru/sale/commercial/277396177/',
 'https://kazan.cian.ru/sale/commercial/280712837/',
 'https://kazan.cian.ru/sale/commercial/279783670/',
 'https://kazan.cian.ru/sale/commercial/260145168/',
 'https://kazan.cian.ru/sale/commercial/281671043/',
 'https://kazan.cian.ru/sale/commercial/279782877/',
 'https://kazan.cian.ru/sale/commercial/282629029/',
 'https://kazan.cian.ru/sale/commercial/276405783/',
 'https://kazan.cian.ru/sale/commercial/277815729/',
 'https://kazan.cian.ru/sale/commercial/284316281/',
 'https://kazan.cian.ru/sale/commercial/276402403/',
 'https://kazan.cian.ru/sale/commercial/281893433/',
 'https://kazan.cian.ru/sale/commercial/279010546/',
 'https://kazan.cian.ru/sale/commercial/279815527/',
 'https://kazan.cian.ru/sale/commercial/284550730/',
 'https://kazan.cian.ru/sale/commercial/276408263/',
 'https://kazan.cian.ru/sale/commercial/275808301/',
 'https://kazan.cian.ru/sale/commercial/276408833/',
 'https://kazan.cian.ru/sale/commercial/276406

In [70]:
len(link_all)

46

In [71]:
#options = webdriver.ChromeOptions()
#options.add_argument("start_maximized")

#options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_experimental_option('useAutomationExtension', False)
#driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
##https://chromedriver.chromium.org/ - тут можно скачать актуальный chromedriver.exe

#stealth(driver,
#        languages=["en-US", "en"],
#        vendor="Google Inc.",
#        platform="Win32",
#        webgl_vendor="Intel Iris OpenGL Engine",
#        fix_hairline=True,
#        )

#url = f"https://kazan.cian.ru/sale/commercial/284551039/"
#driver.get(url)
#sleep(2) #спим 2 секунды (прогрузка страницы)

t1 = [] #список
a = 0

for lnk in tqdm(link_all, 'Сбор данных'):
    try:
        a = a + 1 #счетчик
        t1.clear() #очищаем список значений

        driver.get(lnk) #открываем ссылку
        sleep(2) #спим 2 секунды (прогрузка страницы)

        #Площадь
        inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
        ls_1 = ['Назначение', inf_1.split(',')[0]] #создаем список для назначения
        ls_2 = ['Площадь_1', inf_1.split(' ')[2]] #создаем список для площади из загаловка
        ls_3 = ['ед.изм', inf_1.split(' ')[3]] #создаем список для ед.изм из загаловка
        t1.append(ls_1) #складываем в список
        t1.append(ls_2) #складываем в список
        t1.append(ls_3) #складываем в список

        #Площадь, категория
        inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
        ls_1 = ['Площадь_2', inf_2[0].text.split(' ')[0]] #создаем список для площади из описания
        ls_2 = ['Категория', inf_2[1].text] #создаем список для категории
        t1.append(ls_1) #складываем в список
        t1.append(ls_2) #складываем в список

        #Текст объявления
        inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
        ls_1 = ['Текст_об', inf_3]
        t1.append(ls_1)

        #Адрес
        inf_4 = driver.find_element_by_class_name('a10a3f92e9--address--fHIqz').text
        ls_1 = ['Адрес', inf_4]
        t1.append(ls_1)

        #Технические характеристики
        inf_5_1 = driver.find_elements_by_class_name('a10a3f92e9--name--ZFuu3')
        inf_5_2 = driver.find_elements_by_class_name('a10a3f92e9--value--eZn1s')
        k_1 = len(inf_5_1) #количество строк в таблице
        for j in range(0, k_1):
            table_1 = inf_5_1[j].text #выгружаем названия
            table_2 = inf_5_2[j].text #выгружаем наличие
            ls_1 = [table_1, table_2] #формируем таблицу
            t1.append(ls_1) #складываем в список

        #Стоимость
        price = driver.find_element_by_class_name('a10a3f92e9--price--PcAEt').find_element_by_tag_name('span').get_attribute("content")
        ls_1 = ['Цена', price]
        t1.append(ls_1)
        
        #ССЫЛКА
        lnk_1 = ['Ссылка', lnk] #создаем список для ссылки
        t1.append(lnk_1) #складываем в список

        df = pd.DataFrame(t1)
        df = df.set_index(0)    

        if a == 1:
            df_result = df
        if a > 1:
            df_result = pd.concat([df_result, df], axis = 1)

    except:
        continue

df_result_copy = df_result.copy() #делаем копию данных
df_result_copy = df_result_copy.transpose() #транспонируем таблицу

#Сохраняем в Excel
writer = pd.ExcelWriter('ЗУ_комм_Казань.xlsx')
df_result_copy.to_excel(writer, 'marks')
writer.save()

Сбор данных:   0%|                                                                              | 0/46 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

Сбор данных:  11%|███████▌                                                              | 5/46 [01:25<11:30, 16.85s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:69: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  price = driver.find_element_by_class_name('a10a3f92e9--price--PcAEt').find_element_by_tag_name('span').get_attribute("content")
Сбор данных:  20%|█████████████▋                                                        | 9/46 [01:40<03:59,  6.46s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Us

Сбор данных:  28%|███████████████████▌                                                 | 13/46 [01:55<02:26,  4.45s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

Сбор данных:  37%|█████████████████████████▌                                           | 17/46 [02:13<02:10,  4.50s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

...

Сбор данных:  61%|██████████████████████████████████████████                           | 28/46 [02:51<01:02,  3.46s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

Сбор данных:  70%|████████████████████████████████████████████████                     | 32/46 [03:05<00:48,  3.46s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:59: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_5_1 = driver.find_elements_by_class_name('a10a3f92e9--name--ZFuu3')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:60: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_5_2 = driver.find_elements_by_class_name('a10a3f92e9--value--eZn1s')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:69: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  price = driver.find_element_by_class_name('a10a3f92e9--price--PcAEt').find_element_by_tag_name('span').get_attribute("content")
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:69: DeprecationWarning: find_element_by_tag_name is deprecated. Please use fi

Сбор данных:  85%|██████████████████████████████████████████████████████████▌          | 39/46 [03:30<00:26,  3.77s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a3f92e9--container--PJnny').text #.split(',')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  inf_2 = driver.find_elements_by_class_name('a10a3f92e9--info-value--bm3DC')
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_3 = driver.find_element_by_class_name('a10a3f92e9--description-text--YNzWU').text
C:\Users\skapr\AppData\Local\Temp\ipykernel

C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:69: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  price = driver.find_element_by_class_name('a10a3f92e9--price--PcAEt').find_element_by_tag_name('span').get_attribute("content")
C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:69: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  price = driver.find_element_by_class_name('a10a3f92e9--price--PcAEt').find_element_by_tag_name('span').get_attribute("content")
Сбор данных:  93%|████████████████████████████████████████████████████████████████▌    | 43/46 [03:44<00:11,  3.77s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_26124\2607612282.py:33: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  inf_1 = driver.find_element_by_class_name('a10a